**Dự án này được nấu bởi [Phạm Huỳnh Anh](https://github.com/PhamHuynhAnh16)**

# **Vui lòng không sử dụng dự án với bất kỳ mục đích nào vi phạm đạo đức, pháp luật, hoặc gây tổn hại đến cá nhân, tổ chức...**

# **Trong trường hợp người sử dụng không tuân thủ các điều khoản hoặc vi phạm, tôi sẽ không chịu trách nhiệm về bất kỳ khiếu nại, thiệt hại, hay trách nhiệm pháp lý nào, dù là trong hợp đồng, do sơ suất, hay các lý do khác, phát sinh từ, ngoài, hoặc liên quan đến phần mềm, việc sử dụng phần mềm hoặc các giao dịch khác liên quan đến phần mềm.**

In [ ]:
#@title **🌏 Cài đặt**
import os

from ipywidgets import Button
from IPython.display import clear_output

print("👩🏻‍💻 Cài đặt...")

!git clone https://github.com/PhamHuynhAnh16/Vietnamese-RVC /content/Vietnamese_RVC > /dev/null 2>&1
!pip install -r /content/Vietnamese_RVC/requirements.txt --no-cache-dir -q > /dev/null 2>&1

#@markdown **💻 Cài đặt sẽ mất khoảng 2 phút để hoàn tất!**

os.environ["TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD"] = "1"
os.environ["TORCH_FORCE_WEIGHTS_ONLY_LOAD"] = "0"

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **📱 Mở giao diện sử dụng**
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%cd /content/Vietnamese_RVC
#@markdown **Để trải nghiệm hết tính năng hãy dùng giao diện:) Còn muốn đơn giản thì không dùng giao diện**

#@markdown **Nếu biết có thể sử dụng biểu đồ để kiểm tra huấn luyện quá sức 👍**
su_dung_bieu_do = False #@param {type:"boolean"}

if su_dung_bieu_do:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

!wget https://huggingface.co/AnhP/Vietnamese-RVC-Project/resolve/main/predictors/rmvpe.pt -O /content/Vietnamese_RVC/assets/models/predictors/rmvpe.pt > /dev/null 2>&1
!wget https://huggingface.co/AnhP/Vietnamese-RVC-Project/resolve/main/embedders/fairseq/hubert_base.pt -O /content/Vietnamese_RVC/assets/models/embedders/hubert_base.pt > /dev/null 2>&1

!python main/app/app.py --share

# **Tùy chỉnh thêm 🧰**

In [ ]:
#@title **Kết nối hoặc ngắt kết nối với drive ☁**
import os

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output, display

if os.path.exists("/content/drive"):
    print("🔗 Ngắt kết nối với drive...")
    try:
        drive.flush_and_unmount()
        clear_output()
        display(Button(description="\u2714 Hoàn Tất!", button_style="success"))
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình ngắt kết nối drive: {e}')
else:
    print('🔗 Kết nối với drive...')
    try:
        drive.mount('/content/drive')
        clear_output()
        display(Button(description="\u2714 Hoàn Tất!", button_style="success"))
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

In [ ]:
#@title **Khởi động hoặc ngừng sao lưu 🛠**
import os
import time
import threading
import subprocess

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output

logs_folder, weights_folder, audios_folder = '/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Nếu không tích vào ô nào thì sẽ ngừng kết nối ở phần đó**
khoi_dong_sao_luu_mo_hinh = False #@param {"type":"boolean"}
khoi_dong_sao_luu_am_thanh = False #@param {"type":"boolean"}
#@markdown **Đồng bộ là sẽ đồng bộ các thư mục sao lưu lại, việc thư mục bị xóa mất 1 tệp thì ở thư mục sao lưu tệp đó cũng sẽ bị xóa**
dong_bo_thu_muc = False #@param {"type":"boolean"}

%cd /content/Vietnamese_RVC

class Channel:
    def __init__(self, source, destination, sync_deletions=False, every=60, exclude = None):
        self.source = source
        self.destination = destination
        self.event = threading.Event()
        self.syncing_thread = threading.Thread(target=self._sync, args=())
        self.sync_deletions = sync_deletions
        self.every = every

        if not exclude: exclude = []
        if isinstance(exclude, str): exclude = [exclude]

        self.exclude = exclude
        self.command = ['rsync', '-aP']

    def alive(self):
        if self.syncing_thread.is_alive(): return True
        else: return False

    def _sync(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')

        while not self.event.is_set():
            subprocess.run(command)
            time.sleep(self.every)

    def copy(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')
        subprocess.run(command)

        return True

    def start(self):
        if self.syncing_thread.is_alive():
            self.event.set()
            self.syncing_thread.join()

        if self.event.is_set(): self.event.clear()
        if self.syncing_thread._started.is_set(): self.syncing_thread = threading.Thread(target=self._sync, args=())

        self.syncing_thread.start()
        return self.alive()

    def stop(self):
        if self.alive():
            self.event.set()
            self.syncing_thread.join()

            while self.alive():
                if not self.alive(): break

        return not self.alive()

if not "logs_backup" in locals(): logs_backup = Channel("/content/Vietnamese_RVC/assets/logs", logs_folder, sync_deletions=dong_bo_thu_muc, every=40, exclude="mute")
if not "weights_backup" in locals(): weights_backup = Channel("/content/Vietnamese_RVC/assets/weights", weights_folder, sync_deletions=dong_bo_thu_muc, every=40)
if not "audio_backup" in locals(): audio_backup = Channel("/content/Vietnamese_RVC/audios", audios_folder, sync_deletions=dong_bo_thu_muc, every=40)

logs_backup.stop(); weights_backup.stop(); audio_backup.stop()

if os.path.exists('/content/drive/MyDrive'):
    if khoi_dong_sao_luu_mo_hinh:
        if not os.path.exists(logs_folder): os.makedirs(logs_folder)
        if not os.path.exists(weights_folder): os.makedirs(weights_folder)

        logs_backup.start(); weights_backup.start()
    else: logs_backup.stop(); weights_backup.stop()

    if khoi_dong_sao_luu_am_thanh:
        if not os.path.exists(audios_folder): os.makedirs(audios_folder)
        audio_backup.start()
    else: audio_backup.stop()
else:
    try:
        drive.mount('/content/drive')
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

    if khoi_dong_sao_luu_mo_hinh:
        if not os.path.exists(logs_folder): os.makedirs(logs_folder)
        if not os.path.exists(weights_folder): os.makedirs(weights_folder)

        logs_backup.start(); weights_backup.start()
    else: logs_backup.stop(); weights_backup.stop()

    if khoi_dong_sao_luu_am_thanh:
        if not os.path.exists(audios_folder): os.makedirs(audios_folder)
        audio_backup.start()
    else: audio_backup.stop()

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **♻️ Khởi động dọn rác**
import os
import time
import threading

from ipywidgets import Button
from google.colab import auth, drive
from IPython.display import clear_output

try:
    from googleapiclient.discovery import build
except:
    os.system("pip install google-api-python-client")
    from googleapiclient.discovery import build

class Clean:
    def __init__(self, every=60):
        self.service = build('drive', 'v3')
        self.every = every
        self.trash_cleanup_thread = None

    def delete(self):
        page_token = None

        while 1:
            response = self.service.files().list(q="trashed=true", spaces='drive', fields="nextPageToken, files(id, name)", pageToken=page_token).execute()

            for file in response.get('files', []):
                if file['name'].startswith("G_") and file['name'].endswith(".pth") or file['name'].startswith("D_") and file['name'].endswith(".pth"):
                    try:
                        self.service.files().delete(fileId=file['id']).execute()
                    except Exception as e:
                        raise RuntimeError(e)

            page_token = response.get('nextPageToken', None)
            if page_token is None: break

    def clean(self):
        while 1:
            self.delete()
            time.sleep(self.every)

    def start(self):
        self.trash_cleanup_thread = threading.Thread(target=self.clean)
        self.trash_cleanup_thread.daemon = True
        self.trash_cleanup_thread.start()

    def stop(self):
        if self.trash_cleanup_thread: self.trash_cleanup_thread.join()

%cd /content/Vietnamese_RVC

#@markdown **Sử dụng khi huấn luyện sẽ dọn bớt các tệp tin D, G trong thùng rác google drive**
khoi_dong_don_rac = False #@param {"type":"boolean"}

if khoi_dong_don_rac:
    if os.path.exists('/content/drive/MyDrive'):
        auth.authenticate_user()
    else:
        try:
            drive.mount('/content/drive')
        except Exception as e:
            raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

        auth.authenticate_user()
    Clean(every=40).start()
else: Clean().stop()

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **Tải dữ liệu sao lưu từ drive 📂**
import os
import shutil
from ipywidgets import Button
from IPython.display import clear_output, display

logs_folder, weights_folder, audios_folder ='/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Tải các mô hình huấn luyện để tiếp tục huấn luyện**
tai_mo_hinh = False # @param {"type":"boolean"}
#@markdown **Tải các âm thanh được sao lưu để tiếp tục sử dụng**
tai_am_thanh = False # @param {"type":"boolean"}

if os.path.exists("/content/drive/MyDrive"):
  if tai_mo_hinh:
    if len(os.listdir(logs_folder)) < 1 or len(os.listdir(weights_folder)) < 1: print("Không tìm thấy dữ Liệu")
    else:
      if os.path.exists("/content/drive/MyDrive/model"):
        shutil.copytree(logs_folder, "/content/Vietnamese_RVC/assets/logs", dirs_exist_ok=True)
        shutil.copytree(weights_folder, "/content/Vietnamese_RVC/assets/weights", dirs_exist_ok=True)

        clear_output()
      else: print("Không tìm thấy dữ liệu mô hình")
  if tai_am_thanh:
    if len(os.listdir(audios_folder)) < 1: print("Không tìm thấy dữ Liệu")
    else:
      if os.path.exists("/content/drive/MyDrive/audios"):
        shutil.copytree(audios_folder, "/content/Vietnamese_RVC/audios", dirs_exist_ok=True)
        clear_output()
      else: print("Không tìm thấy dữ liệu âm thanh")
else: print("Google drive không được kết nối")

display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

# **Tải Xuống 📩**

In [ ]:
#@title **🔎 Tìm kiếm mô hình**
import json
import codecs
import requests

from bs4 import BeautifulSoup

def fetch_models_data(search):
    all_table_data = []
    page = 1

    while 1:
        try:
            response = requests.post(url=codecs.decode("uggcf://ibvpr-zbqryf.pbz/srgpu_qngn.cuc", "rot13"), data={"page": page, "search": search})

            if response.status_code == 200:
                table_data = response.json().get("table", "")
                if not table_data.strip(): break
                all_table_data.append(table_data)
                page += 1
            else: raise Exception(f"Đã xảy ra lỗi, Mã lỗi: {response.status_code}")
        except json.JSONDecodeError:
            raise Exception("Đã xảy ra lỗi không thể phân tích từ phản hồi.")
        except requests.RequestException as e:
            raise Exception(f"Gửi yêu cầu thất bại: {e}")
    return all_table_data

def search_models(name):
    if not name: raise NameError("Vui lòng nhập tên mô hình")
    tables = fetch_models_data(name)

    if len(tables) == 0: print("Không tìm thấy...")
    else:
        for table in tables:
            for row in BeautifulSoup(table, "html.parser").select("tr"):
                name_tag, url_tag = row.find("a", {"class": "fs-5"}), row.find("a", {"class": "btn btn-sm fw-bold btn-light ms-0 p-1 ps-2 pe-2"})
                if name_tag and url_tag:
                    name = name_tag.text.replace(".onnx", "").replace(".pth", "").replace(".index", "").replace(".zip", "").replace(" ", "_").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(",", "").replace('"', "").replace("'", "").replace("|", "_").replace("-_-", "_").replace("_-_", "_").replace("-", "_").replace("---", "_").replace("___", "_").strip()
                    url = url_tag["href"].replace("https://easyaivoice.com/run?url=", "")
                    if "huggingface" in url: print(f"{name}: {url}")

#@markdown **Tìm kiếm liên kết mô hình**
ten_mo_hinh = "" # @param {"type":"string","placeholder":"Tên của mô hình cần tìm"}

search_models(ten_mo_hinh)

In [ ]:
#@title **📩 Tải xuống mô hình**
from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

#@markdown **Hỗ trợ các liên kết đến từ huggingface.co / drive.google.com / mega.nz / mediafire.com**

ten_cua_mo_hinh = "" # @param {"type":"string","placeholder":"Tên mô hình"}
lien_ket_mo_hinh = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}

if not lien_ket_mo_hinh:
    uploaded = files.upload()
    args = f'from main.app.core.process import save_drop_model; save_drop_model(\\"{list(uploaded.keys())[0]}\\")'
    !python3 -c "$args"
else:
    args = f'from main.app.core.downloads import download_model; download_model(\\"{lien_ket_mo_hinh}\\", \\"{ten_cua_mo_hinh}\\")'
    !python3 -c "$args"

clear_output()
display(Button(description="\u2714 Hoàn tất!!", button_style="success"))

In [ ]:
#@title 🧾 **Tải xuống mô hình huấn luyện trước**
#@markdown **Chạy ô để có thể lựa chọn mô hình và tốc độ lấy mẫu của mô hình để tiến hành tải xống**
import os
import shutil
import codecs
import requests

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

from main.tools import huggingface

def fetch_pretrained_data():
    response = requests.get(codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/wfba/phfgbz_cergenvarq.wfba", "rot13"))
    response.raise_for_status()
    return response.json()

model_list = list(fetch_pretrained_data().keys())

for m in model_list:
    print(f"{model_list.index(m)}. {m}")

while 1:
    try:
        model_index = int(input("Nhập số thứ tự của mô hình: "))
        if 0 <= model_index < len(model_list):
            selected_model = model_list[model_index]
            clear_output()
            print(f"Bạn đã chọn mô hình: {selected_model}")
            break
        else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
    except ValueError:
        print("Vui lòng nhập một số nguyên.")
    except IndexError:
        print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

model_sr_list = list(fetch_pretrained_data()[selected_model].keys())

if len(model_sr_list) == 1: selected_sr = model_sr_list[0]
else:
    for sr in model_sr_list:
        print(f"{model_sr_list.index(sr)}. {sr}")

    while 1:
        try:
            model_sr_index = int(input("Nhập số thứ tự của tốc độ lấy mẫu: "))
            if 0 <= model_sr_index < len(model_sr_list):
                selected_sr = model_sr_list[model_sr_index]
                print(f"Bạn đã chọn tốc độ lấy mẫu: {selected_sr}")
                clear_output()
                break
            else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
        except ValueError:
            print("Vui lòng nhập một số nguyên.")
        except IndexError:
            print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

paths = fetch_pretrained_data()[selected_model][selected_sr]
pretraineds_custom_path = os.path.join("assets", "models", "pretrained_custom")

if not os.path.exists(pretraineds_custom_path): os.makedirs(pretraineds_custom_path, exist_ok=True)
url = codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/cergenvarq_phfgbz/", "rot13") + paths

file = huggingface.HF_download_file(url.replace("/blob/", "/resolve/").replace("?download=true", "").strip(), os.path.join(pretraineds_custom_path, paths))
if file.endswith(".zip"):
    shutil.unpack_archive(file, pretraineds_custom_path)
    os.remove(file)

clear_output()
Button(description="\u2714 Hoàn tất!", button_style="success")

In [ ]:
#@title 📥 **Tải xuống mô hình huấn luyện trước**
import os
import json

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

#@markdown **Nhập đường dẫn huấn luyện trước để tiến hành tải xuống huấn luyện trước**
huan_luyen_truoc_D = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}
huan_luyen_truoc_G = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}

download_url = "Tải từ đường dẫn liên kết" if json.load(open(os.path.join("main", "configs", "config.json"), "r")).get("language", "vi-VN") == "vi-VN" else "Download from the link"

args = f'from main.app.core.downloads import download_pretrained_model; download_pretrained_model(\\"{download_url}\\", \\"{huan_luyen_truoc_D}\\", \\"{huan_luyen_truoc_G}\\")'
!python3 -c "$args"

clear_output()
Button(description="\u2714 Hoàn tất!", button_style="success")

# **Suy Luận 🔊**

In [ ]:
#@title **Tách Nhạc**
import os
import re
import yt_dlp
import shutil
import warnings

from ipywidgets import Button
from google.colab import files
from urllib.parse import urlparse
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

def download_url(url):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        ydl_opts = {"format": "bestaudio/best", "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "wav", "preferredquality": "192"}], "quiet": True, "no_warnings": True, "noplaylist": True, "verbose": False}

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            audio_output = os.path.join("audios", re.sub(r'\s+', '-', re.sub(r'[^\w\s\u4e00-\u9fff\uac00-\ud7af\u0400-\u04FF\u1100-\u11FF]', '', ydl.extract_info(url, download=False).get('title', 'video')).strip()))
            if os.path.exists(audio_output): shutil.rmtree(audio_output, ignore_errors=True)

            ydl_opts['outtmpl'] = audio_output

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            audio_output += ".wav"
            ydl.download([url])

        return audio_output

def is_url(path):
    try:
        result = urlparse(path)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

#@markdown **Các tùy chọn về tách bè và tách vang khi tách nhạc**
tach_be = False # @param {"type":"boolean"}
tach_vang = False # @param {"type":"boolean"}
#@markdown **Đường dẫn đầu vào (có thể là đường dẫn liên kết hoặc đường dẫn tệp) và định dạng đầu ra tệp**
duong_dan = "" #@param {"type":"string", "placeholder":"Nhập đường dẫn đến tệp âm thanh hoặc đường dẫn liên kết âm thanh"}
dinh_dang_am_thanh = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
#@markdown **Các tùy chọn về mô hình tách nhạc, mức chồng chéo và kích thước phân đoạn**
mo_hinh_tach_nhac = "Voc_FT" #@param ["Main_340", "Main_390", "Main_406", "Main_427", "Main_438", "Inst_full_292", "Inst_HQ_1", "Inst_HQ_2", "Inst_HQ_3", "Inst_HQ_4", "Inst_HQ_5", "Kim_Vocal_1", "Kim_Vocal_2", "Kim_Inst", "Inst_187_beta", "Inst_82_beta", "Inst_90_beta", "Voc_FT", "Crowd_HQ", "Inst_1", "Inst_2", "Inst_3", "MDXNET_1_9703", "MDXNET_2_9682", "MDXNET_3_9662", "Inst_Main", "MDXNET_Main", "MDXNET_9482", "HT-Normal", "HT-Tuned", "HD_MMI",  "HT_6S"]
chong_cheo = "0.25" # @param ["0.25", "0.5", "0.75", "0.99"]
kich_thuoc_phan_doan = 256 # @param {"type":"slider","min":32,"max":2048,"step":8}

if not duong_dan:
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = os.path.join("audios", os.path.basename(filename).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
    shutil.move(filename, input_path)
else: input_path = download_url(duong_dan) if is_url(duong_dan) else duong_dan

args = f'from main.app.core.separate import separator_music; separator_music(\\"{input_path}\\", \\"audios\\", \\"{dinh_dang_am_thanh}\\", 2, {kich_thuoc_phan_doan}, {chong_cheo}, False, 0.7, True, \\"{mo_hinh_tach_nhac}\\", \\"Version-2\\", {tach_be}, {tach_vang}, {tach_be and tach_vang}, 1024, 1, 48000)'
!python3 -c "$args"

clear_output()
print(f"Tệp đầu ra của bạn nằm trong thư mục: /content/Vietnamese_RVC/audios/{os.path.splitext(os.path.basename(input_path))[0]}")
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title 🎵 **Chuyển Đổi Âm Thanh** 🎵
import os
import shutil

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

def convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, input_audio_name, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode, proposal_pitch, proposal_pitch_threshold):
    args = f'from main.app.core.inference import convert_audio; convert_audio({clean}, {autotune}, {use_audio}, {use_original}, {convert_backing}, {not_merge_backing}, {merge_instrument}, {pitch}, {clean_strength}, \\"{model}\\", \\"{index}\\", {index_rate}, \\"{input}\\", \\"{output}\\", \\"{format}\\", \\"{method}\\", \\"{hybrid_method}\\", {hop_length}, \\"{embedders}\\", \\"{custom_embedders}\\", {resample_sr}, {filter_radius}, {volume_envelope}, {protect}, {split_audio}, {f0_autotune_strength}, \\"{input_audio_name}\\", {checkpointing}, {onnx_f0_mode}, {formant_shifting}, {formant_qfrency}, {formant_timbre}, \\"{f0_file}\\", \\"{embedders_mode}\\", {proposal_pitch}, {proposal_pitch_threshold})'
    !python3 -c "$args"

def get_audio_file(output_audio, label):
    matching_files = [f for f in os.listdir(output_audio) if label in f]

    if not matching_files: raise FileNotFoundError("Không tìm thấy nhạc nền!")
    return os.path.join(output_audio, matching_files[0])

def convert_selection(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode, proposal_pitch, proposal_pitch_threshold):
    if use_audio:
        choice = [f for f in os.listdir("audios") if os.path.isdir(os.path.join("audios", f))]

        if len(choice) == 0: raise FileNotFoundError("Không tìm thấy bản tách nhạc nào!")
        elif len(choice) == 1:
            choice_audio = choice[0]
            convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, False, pitch, clean_strength, model, index, index_rate, None, None, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, choice_audio, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode, proposal_pitch, proposal_pitch_threshold)
        else:
            print("Đã tìm thấy nhiều hơn 1 bản tách nhạc, vui lòng chọn bài để tiến hành chuyển đổi!")
            for c in choice:
                print(f"{choice.index(c)}. {c}")

            while 1:
                try:
                    choice_select = int(input("Hãy nhập số thứ tự của bản tách: "))

                    if 0 <= choice_select < len(choice):
                        choice_audio = choice[choice_select]
                        print(f"Bạn đã chọn bản tách: {choice_audio}")
                        break
                    else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
                except ValueError:
                    print("Vui lòng nhập một số nguyên.")
                except IndexError:
                    print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")
            convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, False, pitch, clean_strength, model, index, index_rate, None, None, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, choice_audio, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode, proposal_pitch, proposal_pitch_threshold)
    else: convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, False, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, None, checkpointing, onnx_f0_mode, formant_shifting, formant_qfrency, formant_timbre, f0_file, embedders_mode, proposal_pitch, proposal_pitch_threshold)

    if use_audio: output_path = f"/content/Vietnamese_RVC/audios/{choice_audio}/Vocals+Backing.{format}" if convert_backing else f"/content/Vietnamese_RVC/audios/{choice_audio}/Convert_Vocals.{format}"
    return audio_effects(output_path, os.path.join("audios", choice_audio, f"Vocals+Instruments.{format}") if merge_instrument else os.path.join("audios", choice_audio, f"{os.path.splitext(os.path.basename(output_path))[0]}_Effects.{format}"), format, merge_instrument, get_audio_file(os.path.join("audios", choice_audio), "Instruments.")) if use_audio else f"/content/Vietnamese_RVC/audios/output.{format}"

def audio_effects(input_path, output_path, export_format, merge_instrument, audio_combination_input):
    if not input_path or not os.path.exists(input_path) or os.path.isdir(input_path): raise FileNotFoundError("Đường dẫn tệp đầu vào không hợp lệ!")
    if not output_path: raise ValueError("Đường dẫn tệp đầu ra không hợp lệ!")
    output_dir = os.path.dirname(output_path) or output_path

    if not os.path.exists(output_dir): os.makedirs(output_dir, exist_ok=True)
    if os.path.exists(output_path): os.remove(output_path)

    !python main/inference/audio_effects.py --input_path $input_path --output_path $output_path --reverb_room_size 0.15 --reverb_damping 0.7 --reverb_wet_level 0.2 --reverb_dry_level 0.8 --reverb_width 1.0 --reverb_freeze_mode False --export_format $export_format --reverb True --audio_combination $merge_instrument --audio_combination_input $audio_combination_input
    return output_path

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

#@markdown **Các tùy chọn sử dụng các bản được tách nhạc được tách trước đó**
su_dung_giong_goc = False #@param {"type":"boolean"}
chuyen_doi_giong_be = False #@param {"type":"boolean"}
ket_hop_nhac_nen = False #@param {"type":"boolean"}
#@markdown **Các tùy chọn về mô hình như: Cao độ và Chỉ mục**
ten_mo_hinh = "" #@param {"type":"string","placeholder":"Tên mô hình"}
cao_do = 0 #@param {"type":"slider","min":-20,"max":20,"step":1}
anh_huong_cua_chi_muc = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.01}
#@markdown **Tùy chọn đầu vào và định dạng đầu ra của tệp**
duong_dan_dau_vao = "" #@param {"type":"string","placeholder":"Đường dẫn tệp âm thanh"}
dinh_dang_tep = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
#@markdown **Các tùy chọn về trích xuất dữ liệu, hop length, bảo vệ âm thanh và tự động điều chỉnh trích xuất để đạt chất lượng cao hơn**
phuong_phap_trich_xuat = "rmvpe" #@param ["mangio-crepe-full", "crepe-full", "fcpe", "rmvpe", "harvest", "pyin"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}
bao_ve_phu_am = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.1}
#@markdown **Tùy chọn về cắt âm thanh**
cat_am_thanh = False #@param {"type":"boolean"}

if ten_mo_hinh:
    if not ten_mo_hinh.endswith((".pth", ".onnx")): ten_mo_hinh = ten_mo_hinh + ".pth"
    model_path = os.path.join("assets", "weights", ten_mo_hinh)
    index_path = get_index(ten_mo_hinh.split("_")[0])
else:
    model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith((".pth", ".onnx")) and not model.startswith("G_") and not model.startswith("D_")))
    indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

    if len(model_name) < 1: raise ValueError("Vui lòng cung cấp mô hình để tiến hành chuyển đổi!")
    elif len(model_name) == 1:
        model_path = os.path.join("assets", "weights", model_name[0])
        index_path = get_index(os.path.basename(model_name[0])[0])
    else:
        print("Tìm thấy nhiều hơn 1 mô hình, vui lòng nhập số thứ tự của mô hình để tiến hành chuyển đổi!")
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index = int(input("Nhập số thứ tự của mô hình: "))
                if model_index < len(model_name):
                    selected_model = model_name[model_index]
                    print(f"Bạn đã chọn mô hình: {selected_model}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        model_path = os.path.join("assets", "weights", selected_model)
        index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("Không tìm thấy chỉ mục!")

if not duong_dan_dau_vao and not (su_dung_giong_goc or chuyen_doi_giong_be or ket_hop_nhac_nen):
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = os.path.join("audios", os.path.basename(filename).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
    shutil.move(filename, input_path)
else: input_path = duong_dan_dau_vao

output_files = convert_selection(False, False, (su_dung_giong_goc or chuyen_doi_giong_be or ket_hop_nhac_nen), su_dung_giong_goc, chuyen_doi_giong_be, False, ket_hop_nhac_nen, cao_do, 0.7, model_path, index_path, anh_huong_cua_chi_muc, input_path, os.path.join("audios", "output.wav"), dinh_dang_tep, phuong_phap_trich_xuat, None, hop_length, "hubert_base", None, 0, 3, 1, bao_ve_phu_am, cat_am_thanh, 1, False, False, False, 0, 0, "", "fairseq", False, 255.0)
clear_output()

print(f"Tệp đầu ra của bạn là: {output_files}")
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title 🎵 **Chuyển Đổi Văn Bản** 🎵
import os
import zipfile
import xml.etree.ElementTree

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

def read_docx_text(path):
    with zipfile.ZipFile(path) as docx:
        with docx.open("word/document.xml") as document_xml:
            xml_content = document_xml.read()

    WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'

    paragraphs = []
    for paragraph in xml.etree.ElementTree.XML(xml_content).iter(WORD_NAMESPACE + 'p'):
        texts = [node.text for node in paragraph.iter(WORD_NAMESPACE + 't') if node.text]
        if texts: paragraphs.append(''.join(texts))

    return '\n'.join(paragraphs)

def process_input(file_path):
    if file_path.endswith(".srt"): file_contents = ""
    elif file_path.endswith(".docx"): file_contents = read_docx_text(file_path)
    else:
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                file_contents = file.read()
        except Exception as e:
            file_contents = ""

    return file_contents

#@markdown **Văn bản cần đọc và các tùy chọn về giọng đọc và tốc độ đọc**
van_ban = "" # @param {"type":"string", "placeholder":"Văn bản cần đọc"}
giong_noi = "vi-VN-NamMinhNeural" # @param ["af-ZA-AdriNeural", "af-ZA-WillemNeural", "sq-AL-AnilaNeural", "sq-AL-IlirNeural", "am-ET-AmehaNeural", "am-ET-MekdesNeural", "ar-DZ-AminaNeural", "ar-DZ-IsmaelNeural", "ar-BH-AliNeural", "ar-BH-LailaNeural", "ar-EG-SalmaNeural", "ar-EG-ShakirNeural", "ar-IQ-BasselNeural", "ar-IQ-RanaNeural", "ar-JO-SanaNeural", "ar-JO-TaimNeural", "ar-KW-FahedNeural", "ar-KW-NouraNeural", "ar-LB-LaylaNeural", "ar-LB-RamiNeural", "ar-LY-ImanNeural", "ar-LY-OmarNeural", "ar-MA-JamalNeural", "ar-MA-MounaNeural", "ar-OM-AbdullahNeural", "ar-OM-AyshaNeural", "ar-QA-AmalNeural", "ar-QA-MoazNeural", "ar-SA-HamedNeural", "ar-SA-ZariyahNeural", "ar-SY-AmanyNeural", "ar-SY-LaithNeural", "ar-TN-HediNeural", "ar-TN-ReemNeural", "ar-AE-FatimaNeural", "ar-AE-HamdanNeural", "ar-YE-MaryamNeural", "ar-YE-SalehNeural", "az-AZ-BabekNeural", "az-AZ-BanuNeural", "bn-BD-NabanitaNeural", "bn-BD-PradeepNeural", "bn-IN-BashkarNeural", "bn-IN-TanishaaNeural", "bs-BA-GoranNeural", "bs-BA-VesnaNeural", "bg-BG-BorislavNeural", "bg-BG-KalinaNeural", "my-MM-NilarNeural", "my-MM-ThihaNeural", "ca-ES-EnricNeural", "ca-ES-JoanaNeural", "zh-HK-HiuGaaiNeural", "zh-HK-HiuMaanNeural", "zh-HK-WanLungNeural", "zh-CN-XiaoxiaoNeural", "zh-CN-XiaoyiNeural", "zh-CN-YunjianNeural", "zh-CN-YunxiNeural", "zh-CN-YunxiaNeural", "zh-CN-YunyangNeural", "zh-CN-liaoning-XiaobeiNeural", "zh-TW-HsiaoChenNeural", "zh-TW-YunJheNeural", "zh-TW-HsiaoYuNeural", "zh-CN-shaanxi-XiaoniNeural", "hr-HR-GabrijelaNeural", "hr-HR-SreckoNeural", "cs-CZ-AntoninNeural", "cs-CZ-VlastaNeural", "da-DK-ChristelNeural", "da-DK-JeppeNeural", "nl-BE-ArnaudNeural", "nl-BE-DenaNeural", "nl-NL-ColetteNeural", "nl-NL-FennaNeural", "nl-NL-MaartenNeural", "en-AU-NatashaNeural", "en-AU-WilliamNeural", "en-CA-ClaraNeural", "en-CA-LiamNeural", "en-HK-SamNeural", "en-HK-YanNeural", "en-IN-NeerjaExpressiveNeural", "en-IN-NeerjaNeural", "en-IN-PrabhatNeural", "en-IE-ConnorNeural", "en-IE-EmilyNeural", "en-KE-AsiliaNeural", "en-KE-ChilembaNeural", "en-NZ-MitchellNeural", "en-NZ-MollyNeural", "en-NG-AbeoNeural", "en-NG-EzinneNeural", "en-PH-JamesNeural", "en-PH-RosaNeural", "en-SG-LunaNeural", "en-SG-WayneNeural", "en-ZA-LeahNeural", "en-ZA-LukeNeural", "en-TZ-ElimuNeural", "en-TZ-ImaniNeural", "en-GB-LibbyNeural", "en-GB-MaisieNeural", "en-GB-RyanNeural", "en-GB-SoniaNeural", "en-GB-ThomasNeural", "en-US-AvaMultilingualNeural", "en-US-AndrewMultilingualNeural", "en-US-EmmaMultilingualNeural", "en-US-BrianMultilingualNeural", "en-US-AvaNeural", "en-US-AndrewNeural", "en-US-EmmaNeural", "en-US-BrianNeural", "en-US-AnaNeural", "en-US-AriaNeural", "en-US-ChristopherNeural", "en-US-EricNeural", "en-US-GuyNeural", "en-US-JennyNeural", "en-US-MichelleNeural", "en-US-RogerNeural", "en-US-SteffanNeural", "et-EE-AnuNeural", "et-EE-KertNeural", "fil-PH-AngeloNeural", "fil-PH-BlessicaNeural", "fi-FI-HarriNeural", "fi-FI-NooraNeural", "fr-BE-CharlineNeural", "fr-BE-GerardNeural", "fr-CA-ThierryNeural", "fr-CA-AntoineNeural", "fr-CA-JeanNeural", "fr-CA-SylvieNeural", "fr-FR-VivienneMultilingualNeural", "fr-FR-RemyMultilingualNeural", "fr-FR-DeniseNeural", "fr-FR-EloiseNeural", "fr-FR-HenriNeural", "fr-CH-ArianeNeural", "fr-CH-FabriceNeural", "gl-ES-RoiNeural", "gl-ES-SabelaNeural", "ka-GE-EkaNeural", "ka-GE-GiorgiNeural", "de-AT-IngridNeural", "de-AT-JonasNeural", "de-DE-SeraphinaMultilingualNeural", "de-DE-FlorianMultilingualNeural", "de-DE-AmalaNeural", "de-DE-ConradNeural", "de-DE-KatjaNeural", "de-DE-KillianNeural", "de-CH-JanNeural", "de-CH-LeniNeural", "el-GR-AthinaNeural", "el-GR-NestorasNeural", "gu-IN-DhwaniNeural", "gu-IN-NiranjanNeural", "he-IL-AvriNeural", "he-IL-HilaNeural", "hi-IN-MadhurNeural", "hi-IN-SwaraNeural", "hu-HU-NoemiNeural", "hu-HU-TamasNeural", "is-IS-GudrunNeural", "is-IS-GunnarNeural", "id-ID-ArdiNeural", "id-ID-GadisNeural", "ga-IE-ColmNeural", "ga-IE-OrlaNeural", "it-IT-GiuseppeNeural", "it-IT-DiegoNeural", "it-IT-ElsaNeural", "it-IT-IsabellaNeural", "ja-JP-KeitaNeural", "ja-JP-NanamiNeural", "jv-ID-DimasNeural", "jv-ID-SitiNeural", "kn-IN-GaganNeural", "kn-IN-SapnaNeural", "kk-KZ-AigulNeural", "kk-KZ-DauletNeural", "km-KH-PisethNeural", "km-KH-SreymomNeural", "ko-KR-HyunsuNeural", "ko-KR-InJoonNeural", "ko-KR-SunHiNeural", "lo-LA-ChanthavongNeural", "lo-LA-KeomanyNeural", "lv-LV-EveritaNeural", "lv-LV-NilsNeural", "lt-LT-LeonasNeural", "lt-LT-OnaNeural", "mk-MK-AleksandarNeural", "mk-MK-MarijaNeural", "ms-MY-OsmanNeural", "ms-MY-YasminNeural", "ml-IN-MidhunNeural", "ml-IN-SobhanaNeural", "mt-MT-GraceNeural", "mt-MT-JosephNeural", "mr-IN-AarohiNeural", "mr-IN-ManoharNeural", "mn-MN-BataaNeural", "mn-MN-YesuiNeural", "ne-NP-HemkalaNeural", "ne-NP-SagarNeural", "nb-NO-FinnNeural", "nb-NO-PernilleNeural", "ps-AF-GulNawazNeural", "ps-AF-LatifaNeural", "fa-IR-DilaraNeural", "fa-IR-FaridNeural", "pl-PL-MarekNeural", "pl-PL-ZofiaNeural", "pt-BR-ThalitaNeural", "pt-BR-AntonioNeural", "pt-BR-FranciscaNeural", "pt-PT-DuarteNeural", "pt-PT-RaquelNeural", "ro-RO-AlinaNeural", "ro-RO-EmilNeural", "ru-RU-DmitryNeural", "ru-RU-SvetlanaNeural", "sr-RS-NicholasNeural", "sr-RS-SophieNeural", "si-LK-SameeraNeural", "si-LK-ThiliniNeural", "sk-SK-LukasNeural", "sk-SK-ViktoriaNeural", "sl-SI-PetraNeural", "sl-SI-RokNeural", "so-SO-MuuseNeural", "so-SO-UbaxNeural", "es-AR-ElenaNeural", "es-AR-TomasNeural", "es-BO-MarceloNeural", "es-BO-SofiaNeural", "es-CL-CatalinaNeural", "es-CL-LorenzoNeural", "es-ES-XimenaNeural", "es-CO-GonzaloNeural", "es-CO-SalomeNeural", "es-CR-JuanNeural", "es-CR-MariaNeural", "es-CU-BelkysNeural", "es-CU-ManuelNeural", "es-DO-EmilioNeural", "es-DO-RamonaNeural", "es-EC-AndreaNeural", "es-EC-LuisNeural", "es-SV-LorenaNeural", "es-SV-RodrigoNeural", "es-GQ-JavierNeural", "es-GQ-TeresaNeural", "es-GT-AndresNeural", "es-GT-MartaNeural", "es-HN-CarlosNeural", "es-HN-KarlaNeural", "es-MX-DaliaNeural", "es-MX-JorgeNeural", "es-NI-FedericoNeural", "es-NI-YolandaNeural", "es-PA-MargaritaNeural", "es-PA-RobertoNeural", "es-PY-MarioNeural", "es-PY-TaniaNeural", "es-PE-AlexNeural", "es-PE-CamilaNeural", "es-PR-KarinaNeural", "es-PR-VictorNeural", "es-ES-AlvaroNeural", "es-ES-ElviraNeural", "es-US-AlonsoNeural", "es-US-PalomaNeural", "es-UY-MateoNeural", "es-UY-ValentinaNeural", "es-VE-PaolaNeural", "es-VE-SebastianNeural", "su-ID-JajangNeural", "su-ID-TutiNeural", "sw-KE-RafikiNeural", "sw-KE-ZuriNeural", "sw-TZ-DaudiNeural", "sw-TZ-RehemaNeural", "sv-SE-MattiasNeural", "sv-SE-SofieNeural", "ta-IN-PallaviNeural", "ta-IN-ValluvarNeural", "ta-MY-KaniNeural", "ta-MY-SuryaNeural", "ta-SG-AnbuNeural", "ta-SG-VenbaNeural", "ta-LK-KumarNeural", "ta-LK-SaranyaNeural", "te-IN-MohanNeural", "te-IN-ShrutiNeural", "th-TH-NiwatNeural", "th-TH-PremwadeeNeural", "tr-TR-AhmetNeural", "tr-TR-EmelNeural", "uk-UA-OstapNeural", "uk-UA-PolinaNeural", "ur-IN-GulNeural", "ur-IN-SalmanNeural", "ur-PK-AsadNeural", "ur-PK-UzmaNeural", "uz-UZ-MadinaNeural", "uz-UZ-SardorNeural", "vi-VN-HoaiMyNeural", "vi-VN-NamMinhNeural", "cy-GB-AledNeural", "cy-GB-NiaNeural", "zu-ZA-ThandoNeural", "zu-ZA-ThembaNeural"]
toc_do_doc = 0 # @param {"type":"slider","min":-100,"max":100,"step":1}
#@markdown **Các tùy chọn về mô hình như: Cao độ và Chỉ mục**
ten_mo_hinh = "" #@param {"type":"string","placeholder":"Tên mô hình"}
cao_do = 0 #@param {"type":"slider","min":-20,"max":20,"step":1}
anh_huong_cua_chi_muc = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.01}
#@markdown **Các tùy chọn về trích xuất dữ liệu, hop length, bảo vệ âm thanh**
phuong_phap_trich_xuat = "rmvpe" #@param ["mangio-crepe-full", "crepe-full", "fcpe", "rmvpe", "harvest", "pyin"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}
bao_ve_phu_am = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.1}
#@markdown **Tùy chọn về cắt âm thanh tĩnh để đạt tốc độ cao hơn và định dạng tệp đầu ra**
cat_am_thanh = False #@param {"type":"boolean"}
dinh_dang_tep = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]

filename = None
if van_ban: input_text = van_ban
else:
    print("Ô văn bản trống. Vui lòng tải lên 1 tệp văn bản txt để tiến hành chuyển đổi!")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_text = process_input(filename)

if ten_mo_hinh:
    if not ten_mo_hinh.endswith((".pth", ".onnx")): ten_mo_hinh = ten_mo_hinh + ".pth"
    model_path = os.path.join("assets", "weights", ten_mo_hinh)
    index_path = get_index(f"{ten_mo_hinh}.pth".split("_")[0])
else:
    model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith((".pth", ".onnx")) and not model.startswith("G_") and not model.startswith("D_")))
    indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

    if len(model_name) < 1: raise ValueError("Vui lòng cung cấp mô hình để tiến hành chuyển đổi!")
    elif len(model_name) == 1:
        model_path = os.path.join("assets", "weights", model_name[0])
        index_path = get_index(os.path.basename(model_name[0])[0])
    else:
        print("Tìm thấy nhiều hơn 1 mô hình, vui lòng nhập số thứ tự của mô hình để tiến hành chuyển đổi!")
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index = int(input("Nhập số thứ tự của mô hình: "))
                if 0 <= model_index < len(model_name):
                    selected_model = model_name[model_index]
                    print(f"Bạn đã chọn mô hình: {selected_model}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        model_path = os.path.join("assets", "weights", selected_model)
        index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("Không tìm thấy chỉ mục!")

tts_output = os.path.join("audios", f"tts.{dinh_dang_tep}")
convert_tts_output = os.path.join("audios", f"tts-convert.{dinh_dang_tep}")

args = f'from main.app.core.tts import TTS; TTS(\\"{input_text}\\", \\"{giong_noi}\\", {toc_do_doc}, \\"{tts_output}\\", 0, False, \\"{filename}\\")'
!python3 -c "$args"

args = f'from main.app.core.inference import convert_tts; convert_tts(False, False, {cao_do}, 0.7, \\"{model_path}\\", \\"{index_path}\\", {anh_huong_cua_chi_muc}, \\"{tts_output}\\", \\"{convert_tts_output}\\", \\"{dinh_dang_tep}\\", \\"{phuong_phap_trich_xuat}\\", None, {hop_length}, \\"hubert_base\\", None, 0, 3, 1, {bao_ve_phu_am}, {cat_am_thanh}, 1, False, False, False, 0, 0, \\"None\\", \\"fairseq\\", False, 255.0)'
!python3 -c "$args"

clear_output()
print(f"Tệp đầu ra của bạn là: /content/Vietnamese_RVC/audios/tts-convert.{dinh_dang_tep}")
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

# **Huấn Luyện Mô Hình 🤖**

In [ ]:
#@title 📁 **Tạo dữ liệu huấn luyện từ đường dẫn**
from ipywidgets import Button
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

#@markdown **Tạo dữ liệu huấn luyện từ đường dẫn liên kết**
lam_sach_du_lieu = False #@param {"type":"boolean"}
duong_dan_lien_ket = "" #@param {"type":"string", placeholder:"Đường dẫn liên kết"}

args = f'from main.app.core.training import create_dataset; [None for _ in create_dataset(\\"{duong_dan_lien_ket}\\", \\"dataset\\", {lam_sach_du_lieu}, 0.7, True, \\"Version-2\\", 0.25, 384, True, False, 0, 0, 1024, 1, 48000)]'
!python3 -c "$args"

clear_output()
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title 🔨 **Xử lý trích xuất dữ liệu** ⛏️
import os
import shutil

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

#@markdown **Tải dữ liệu từ google drive lên dữ liệu của google colab để tiến hành xử lý và trích xuất**
tai_du_lieu_tu_drive = False # @param {"type":"boolean"}
#@markdown **Tên của mô hình để tiến hành**
ten_mo_hinh = "" #@param {"type":"string", placeholder: "Tên mô hình"}
#@markdown **Tốc độ lấy mẫu của mô hình để tiến hành**
toc_do_lay_mau = "48k" #@param ["32k", "40k", "48k"]
#@markdown **Phương pháp trích xuất cao độ và hop length cho mô hình**
phuong_phap_trich_xuat = "rmvpe" #@param ["mangio-crepe-full", "crepe-full", "fcpe", "rmvpe", "harvest", "pyin"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}

if tai_du_lieu_tu_drive:
    if not os.path.exists("/content/drive/MyDrive"): raise ValueError("Bạn chưa kết nối với google drive")
    if len([f for f in os.listdir("/content/drive/MyDrive/dataset") if not ".ipynb_checkpoints" in f]) < 1: raise FileNotFoundError("Không tìm thấy dữ liệu")

    for audios in os.listdir("/content/drive/MyDrive/dataset"):
        shutil.copy(f"/content/drive/MyDrive/dataset/{audios}", "/content/Vietnamese_RVC/dataset")
elif not any(f.lower().endswith(("wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3")) for f in os.listdir("/content/Vietnamese_RVC/dataset") if os.path.isfile(os.path.join("/content/Vietnamese_RVC/dataset", f))):
    uploaded = files.upload()
    for f in list(uploaded.keys()):
        input_path = os.path.join("dataset", os.path.basename(f).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
        shutil.move(f, input_path)

if not ten_mo_hinh: raise ValueError("Vui lòng cung cấp tên để tiến hành!")

args = f'from main.app.core.training import preprocess; [None for _ in preprocess(\\"{ten_mo_hinh}\\", \\"{toc_do_lay_mau}\\", 2, True, True, \\"dataset\\", False, 0.7)]'
!python3 -c "$args"

args = f'from main.app.core.training import extract; [None for _ in extract(\\"{ten_mo_hinh}\\", \\"v2\\", \\"{phuong_phap_trich_xuat}\\", True, {hop_length}, 2, 0, \\"{toc_do_lay_mau}\\", \\"hubert_base\\", None, False, \\"fairseq\\", False, 1, None, False)]'
!python3 -c "$args"

args = f'from main.app.core.training import create_index; [None for _ in create_index(\\"{ten_mo_hinh}\\", \\"v2\\", \\"Auto\\")]'
!python3 -c "$args"

clear_output()
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title 🤖 **Huấn luyện mô hình**
import os

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

#@markdown **Tên của mô hình và tốc độ lấy mẫu để tiến hành huấn luyện mô hình**
ten_mo_hinh = "" #@param {"type":"string", placeholder: "Tên mô hình"}
toc_do_lay_mau = "48k" #@param ["32k", "40k", "48k"]
#@markdown **Số lượng kỷ nguyên của mô hình và bao nhiêu kỷ nguyên sẽ lưu một lần**
so_luong_ky_nguyen = 300 #@param {"type":"slider", min:1, max:10000}
tan_suat_luu = 50 #@param {"type":"slider", min:1, max:10000}
#@markdown **Tùy chọn về kích thước lô và có sử dụng bộ nhớ đệm hay không**
kich_thuoc_lo = 10 #@param {"type":"slider", min:1, max:20}
bo_nho_dem = False #@param {"type":"boolean"}
#@markdown **Tùy chọn về biểu đồ huấn luyện, tự động kiểm tra huấn luyện quá sức và sử dụng tùy chỉnh mô hình huấn luyện trước**
su_dung_bieu_do = False #@param {type:"boolean"}
kiem_tra_huan_luyen = False #@param {"type":"boolean"}
tuy_chinh_huan_luyen_truoc = False #@param {"type":"boolean"}

if su_dung_bieu_do:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

if tuy_chinh_huan_luyen_truoc:
    model_name = [f for f in os.listdir(os.path.join("assets", "models", "pretrained_custom")) if f.endswith(".pth")]

    if len(model_name) == 2:
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index_d = int(input("Nhập số thứ tự của mô hình D: "))
                if 0 <= model_index_d < len(model_name):
                    selected_model_D = model_name[model_index_d]
                    print(f"Bạn đã chọn mô hình D: {selected_model_D}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        model_name.remove(selected_model_D)
        model_index_g = model_name[0]
    elif len(model_name) >= 2:
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index_d = int(input("Nhập số thứ tự của mô hình D: "))
                if 0 <= model_index_d < len(model_name):
                    selected_model_D = model_name[model_index_d]
                    print(f"Bạn đã chọn mô hình D: {selected_model_D}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        while 1:
            try:
                model_index_g = int(input("Nhập số thứ tự của mô hình G: "))
                if 0 <= model_index_g < len(model_name):
                    selected_model_G = model_name[model_index_g]
                    print(f"Bạn đã chọn mô hình G: {selected_model_G}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        if selected_model_D == selected_model_G: raise ValueError("Mô hình D và G giống nhau!")
        clear_output()
    else: print("không tìm thấy mô hình huấn luyện trước nào!")
else: selected_model_G = selected_model_D = None


args = f'from main.app.core.training import training; [None for _ in training(\\"{ten_mo_hinh}\\", \\"v2\\", {tan_suat_luu}, True, True, {so_luong_ky_nguyen}, \\"{toc_do_lay_mau}\\", {kich_thuoc_lo}, 0, True, False, {tuy_chinh_huan_luyen_truoc}, \\"{selected_model_G}\\", \\"{selected_model_D}\\", {kiem_tra_huan_luyen}, 50, False, {bo_nho_dem}, \\"\\", \\"Default\\", False, False, True, \\"AdamW\\", False)]'
!python3 -c "$args"

Button(description="\u2714 Hoàn tất!", button_style="success")

In [ ]:
#@title 📦 **Nén và lưu mô hình** 📦

#@markdown **Chạy ô này và chọn tệp mô hình cần nén!**

import os
import zipfile

from ipywidgets import Button
from IPython.display import clear_output

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith((".pth", ".onnx")) and not model.startswith("G_") and not model.startswith("D_")))
indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

if len(model_name) < 1: raise ValueError("Vui lòng cung cấp mô hình để tiến hành chuyển đổi!")
elif len(model_name) == 1:
    model_path = os.path.join("assets", "weights", model_name[0])
    index_path = get_index(os.path.basename(model_name[0])[0])
else:
    print("Tìm thấy nhiều hơn 1 mô hình, vui lòng nhập số thứ tự của mô hình để tiến hành chuyển đổi!")
    for m in model_name:
        print(f"{model_name.index(m)}. {m}")

    while 1:
        try:
            model_index = int(input("Nhập số thứ tự của mô hình: "))
            if 0 <= model_index < len(model_name):
                selected_model = model_name[model_index]
                print(f"Bạn đã chọn mô hình: {selected_model}")
                break
            else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
        except ValueError:
            print("Vui lòng nhập một số nguyên.")
        except IndexError:
            print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

    model_path = os.path.join("assets", "weights", selected_model)
    index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("Không tìm thấy chỉ mục!")
zip_file_path = os.path.join("assets", os.path.basename(model_path).split("_")[0] + ".zip")

with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(model_path, os.path.basename(model_path))
    if index_path: zipf.write(index_path, os.path.basename(index_path))

clear_output()
print(f"Đường dẫn mô hình của bạn: {zip_file_path}")
Button(description="\u2714 Hoàn tất!", button_style="success")